<a href="https://colab.research.google.com/github/mitali-datascientist/Coursera_Capstone/blob/master/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part - 3

In [0]:
#Import libraries 
import folium 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import requests
%matplotlib inline

In [0]:
## Read file 
df = pd.read_csv('/content/Toronto')


In [0]:
df.drop(df.columns[0], axis=1, inplace=True)


In [5]:
## View data 
df.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [6]:
##Plot all postcodes 

from folium.plugins import MarkerCluster

toronto_coord = (43.6532, -79.3832)

map = folium.Map(toronto_coord, zoom_start=12)
cluster = MarkerCluster()

for i, row in df.iterrows():
    
    label = '{} {}'.format(row['Postcode'], row['Neighbourhood'])
    marker = folium.Marker(
        location=(row['Latitude'], row['Longitude']),
        popup=label        
    )
    cluster.add_child(marker)

map.add_child(cluster)

In [0]:
radius = 500
LIMIT = 500
CLIENT_ID = '2LYVWNTR3QXP2TGKQIY4AJKH0RJDSNUWYCBZNR1PTUSYSD5M'
CLIENT_SECRET = 'VCMSIKJC5NDAZAX2FI5NXLACNB0CR2TRKMCF0L1HSIEAVP0I' 
VERSION = '20180605'

#function from last lab to get nearby venus
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
## Get all venues in Borough
toronto_df = df[df['Borough'].str.contains('Toronto')]
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676531,-79.295425
41,M4K,East Toronto,"The Danforth West, Riverdale",43.683178,-79.355105
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314667
43,M4M,East Toronto,Studio District,43.660629,-79.334855
44,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133


In [9]:
toronto_df['Borough'].unique()


array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [10]:
toronto_venues_df = getNearbyVenues(toronto_df['Neighbourhood'], toronto_df['Latitude'], toronto_df['Longitude'])


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [11]:
print(toronto_venues_df.shape)
toronto_venues_df.head()

(1777, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.295425,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676531,-79.295425,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676531,-79.295425,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676531,-79.295425,Beaches Rec Centre,43.673659,-79.298634,College Gym
4,The Beaches,43.676531,-79.295425,Upper Beaches,43.680563,-79.292869,Neighborhood


In [12]:
toronto_venues_df.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,61,61,61,61,61,61
"Brockton, Exhibition Place, Parkdale Village",69,69,69,69,69,69
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",71,71,71,71,71,71
"Cabbagetown, St. James Town",41,41,41,41,41,41
Central Bay Street,91,91,91,91,91,91
"Chinatown, Grange Park, Kensington Market",75,75,75,75,75,75
Christie,11,11,11,11,11,11


In [13]:
print('Number of unique venue categories: {}'.format(len(toronto_venues_df['Venue Category'].unique())))


Number of unique venue categories: 226


In [14]:
toronto_onehot = pd.get_dummies(toronto_venues_df[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues_df['Neighborhood']
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,...,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
toronto_onehot.shape


(1777, 226)

In [16]:
toronto_grouped_df = toronto_onehot.groupby('Neighborhood').mean().reset_index()    #this will be the input to kmeans 
print(toronto_grouped_df.shape)
toronto_grouped_df.head()

(38, 226)


,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,...,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.03,0.0,0.010000,0.000000,0.030000,0.0,0.000000,0.0,0.000000,0.020000,0.0,0.030000,0.000000,0.010000,0.0,0.0,0.000000,0.0,0.0,0.02,0.0,0.000000,0.01,0.030000,0.000000,0.0,0.01,0.020000,0.01,0.0,0.000000,0.000000,0.060000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.020000,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.00,0.000000,0.010000,0.000000,0.0,0.030000,0.0,0.000000,0.020000,0.0,0.00,0.000000,0.0,0.0,0.000000,0.010000,0.000000,0.020000,0.010000,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.000000,0.01,0.0,0.0,0.01
1,Berczy Park,0.000000,0.0,0.00,0.0,0.016393,0.000000,0.000000,0.0,0.016393,0.0,0.016393,0.032787,0.0,0.000000,0.016393,0.032787,0.0,0.0,0.016393,0.0,0.0,0.00,0.0,0.000000,0.00,0.032787,0.000000,0.0,0.00,0.000000,0.00,0.0,0.000000,0.016393,0.032787,0.0,0.0,0.000000,0.032787,...,0.000000,0.0,0.032787,0.0,0.016393,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.016393,0.0,0.016393,0.0,0.000000,0.000000,0.0,0.00,0.016393,0.0,0.0,0.000000,0.000000,0.000000,0.016393,0.000000,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,0.000000,0.00,0.0,0.0,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.00,0.0,0.014493,0.014493,0.000000,0.0,0.000000,0.0,0.000000,0.043478,0.0,0.057971,0.000000,0.014493,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.014493,0.00,0.028986,0.000000,0.0,0.00,0.014493,0.00,0.0,0.014493,0.000000,0.057971,0.0,0.0,0.000000,0.000000,...,0.014493,0.0,0.014493,0.0,0.000000,0.000000,0.0,0.0,0.014493,0.00,0.014493,0.000000,0.000000,0.0,0.000000,0.0,0.028986,0.000000,0.0,0.00,0.000000,0.0,0.0,0.014493,0.014493,0.014493,0.000000,0.014493,0.0,0.0,0.0,0.0,0.0,0.028986,0.0,0.014493,0.00,0.0,0.0,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0,0.02,0.0,0.000000,0.010000,0.010000,0.0,0.000000,0.0,0.000000,0.010000,0.0,0.050000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.01,0.010000,0.000000,0.0,0.00,0.000000,0.01,0.0,0.000000,0.000000,0.030000,0.0,0.0,0.000000,0.000000,...,0.010000,0.0,0.030000,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.01,0.010000,0.010000,0.000000,0.0,0.030000,0.0,0.000000,0.030000,0.0,0.01,0.000000,0.0,0.0,0.000000,0.010000,0.000000,0.020000,0.020000,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.000000,0.00,0.0,0.0,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.014085,0.0,0.00,0.0,0.000000,0.000000,0.014085,0.0,0.000000,0.0,0.000000,0.028169,0.0,0.042254,0.000000,0.014085,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.00,0.000000,0.014085,0.0,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.042254,0.0,0.0,0.014085,0.000000,...,0.028169,0.0,0.000000,0.0,0.000000,0.014085,0.0,0.0,0.000000,0.00,0.028169,0.014085,0.000000,0.0,0.000000,0.0,0.000000,0.014085,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.014085,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.00


In [0]:
## Most common venue categories 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venu'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped_df['Neighborhood']

for ind in np.arange(toronto_grouped_df.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped_df.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venu,5th Most Common Venu,6th Most Common Venu,7th Most Common Venu,8th Most Common Venu,9th Most Common Venu,10th Most Common Venu
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Restaurant,Gym,Steakhouse,Asian Restaurant,Bar,Breakfast Spot,American Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Farmers Market,Breakfast Spot,Bakery,Cheese Shop,Lounge,Café,Seafood Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Bar,Café,Bakery,Restaurant,Furniture / Home Store,Breakfast Spot,Supermarket,Vegetarian / Vegan Restaurant,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,Restaurant,Hotel,Pizza Place,Steakhouse,Sushi Restaurant,Seafood Restaurant,Café,Fast Food Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Restaurant,Gym / Fitness Center,Café,Bar,Bakery,Pizza Place,Spa,Electronics Store


##### Clustering Neighborhood 

In [0]:
from sklearn.cluster import KMeans

num_clusters = 5

toronto_grouped_clustering_df = toronto_grouped_df.drop('Neighborhood', axis=1)
kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(toronto_grouped_clustering_df)

In [0]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [0]:
toronto_merged_df = toronto_df.reset_index(drop=True)


In [0]:
toronto_merged_df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)


In [23]:
toronto_merged_df.head()


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676531,-79.295425
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683178,-79.355105
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314667
3,M4M,East Toronto,Studio District,43.660629,-79.334855
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133


In [24]:
toronto_merged_df = toronto_merged_df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venu,5th Most Common Venu,6th Most Common Venu,7th Most Common Venu,8th Most Common Venu,9th Most Common Venu,10th Most Common Venu
0,M4E,East Toronto,The Beaches,43.676531,-79.295425,2.0,Health Food Store,College Gym,Trail,Pub,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683178,-79.355105,2.0,Bus Line,Grocery Store,Park,Ice Cream Shop,Discount Store,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314667,2.0,Park,Pizza Place,Sandwich Place,Fruit & Vegetable Store,Pet Store,Movie Theater,Pub,Fast Food Restaurant,Liquor Store,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.660629,-79.334855,2.0,Diner,Italian Restaurant,Brewery,Sushi Restaurant,American Restaurant,Café,Arts & Crafts Store,Bar,Pizza Place,Coffee Shop
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133,0.0,Bus Line,Swim School,Women's Store,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [0]:
toronto_merged_df.dropna(axis=0, inplace=True)


In [26]:
toronto_merged_df.shape


(38, 16)

In [27]:
## Plot the venues with cluster  color 
import matplotlib.cm as cm 
import matplotlib.colors as colors

map_clusters = folium.Map(location=toronto_coord, zoom_start=12)

x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster_label in zip(toronto_merged_df['Latitude'], toronto_merged_df['Longitude'], toronto_merged_df['Neighborhood'], toronto_merged_df['Cluster Labels']):
    cluster_label = int(cluster_label)
    label = folium.Popup(str(poi) + ' Cluster' + str(cluster_label), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster_label-1],
        fill=True,
        fill_color=rainbow[cluster_label-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters